In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [16]:
data = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_2/train_test_merged.csv")

In [17]:
data

,Campagne,Region,Site,Famille,Variete,Num Parcelle,CodeTracabilite,Date,Tonnage,is_train,Recolte,Summed_Irrigation,Total_N,Total_P,Total_K
0,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10001,TR21010,2020-12-01,1847,True,0,37715.1,287.145757,281.399078,782.276993
1,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10001,TR21010,2021-01-01,"2328,324585",True,1,57936.9,287.145757,281.399078,782.276993
2,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10001,TR21010,2021-02-01,"7473,99536",True,2,75728.4,287.145757,281.399078,782.276993
3,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10001,TR21010,2021-03-01,"19828,1",True,3,106078.8,708.002903,373.085811,927.583113
4,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10003,TR22050,2020-12-01,"15284,9",True,0,13726.5,41.313328,189.180149,313.334136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,22/23,SOUSS,Ouled Abbou,SPA,SPA8,21061,TR21061,2023-03-01,NaN,False,1,0.0,0.000000,0.000000,0.000000
3896,22/23,SOUSS,Ouled Abbou,SPA,SPA8,21061,TR21061,2023-04-01,NaN,False,2,0.0,0.000000,0.000000,0.000000
3897,22/23,SOUSS,Ouled Abbou,SPA,SPA8,21061,TR21061,2023-05-01,NaN,False,3,0.0,0.000000,0.000000,0.000000
3898,22/23,SOUSS,Ouled Abbou,SPA,SPA9,21091,TR21091,2022-12-01,"4858,7",True,0,0.0,0.000000,0.000000,0.000000


In [18]:
data.isna().sum()

Campagne               0
Region                 0
Site                   0
Famille                0
Variete                0
Num Parcelle           0
CodeTracabilite        0
Date                   0
Tonnage              781
is_train               0
Recolte                0
Summed_Irrigation      0
Total_N                0
Total_P                0
Total_K                0
dtype: int64

In [19]:
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True, errors='coerce')

In [20]:
data.dtypes

Campagne                     object
Region                       object
Site                         object
Famille                      object
Variete                      object
Num Parcelle                 object
CodeTracabilite              object
Date                 datetime64[ns]
Tonnage                      object
is_train                       bool
Recolte                       int64
Summed_Irrigation           float64
Total_N                     float64
Total_P                     float64
Total_K                     float64
dtype: object

In [21]:
data.columns

Index(['Campagne', 'Region', 'Site', 'Famille', 'Variete', 'Num Parcelle',
       'CodeTracabilite', 'Date', 'Tonnage', 'is_train', 'Recolte',
       'Summed_Irrigation', 'Total_N', 'Total_P', 'Total_K'],
      dtype='object')

### Preprocessing

In [44]:
train_test = data[data['is_train'] == True]

In [45]:
categorical_features = ['Region', 'Site', 'Famille', 'Variete', 'Campagne', 'Num Parcelle', 'CodeTracabilite', 'Recolte']  # Update as per your dataset

# Parse dates
train_test['Date'] = pd.to_datetime(train_test['Date'])
train_test['Year'] = train_test['Date'].dt.year
train_test['Month'] = train_test['Date'].dt.month
train_test['Day'] = train_test['Date'].dt.day

# Drop the original 'Date' column if it's no longer needed
train_test.drop('Date', axis=1, inplace=True)

C:\Users\DataScience\AppData\Local\Temp\ipykernel_66892\2562847410.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test['Date'] = pd.to_datetime(train_test['Date'])
C:\Users\DataScience\AppData\Local\Temp\ipykernel_66892\2562847410.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test['Year'] = train_test['Date'].dt.year
C:\Users\DataScience\AppData\Local\Temp\ipykernel_66892\2562847410.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [46]:
train_test['Tonnage'] =train_test['Tonnage'].str.replace(',', '.')
train_test['Tonnage'] =train_test['Tonnage'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

C:\Users\DataScience\AppData\Local\Temp\ipykernel_66892\1434961509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test['Tonnage'] =train_test['Tonnage'].str.replace(',', '.')
C:\Users\DataScience\AppData\Local\Temp\ipykernel_66892\1434961509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test['Tonnage'] =train_test['Tonnage'].str.replace(r'[^\d.-]', '', regex=True).astype(float)


In [47]:
train_test

,Campagne,Region,Site,Famille,Variete,Num Parcelle,CodeTracabilite,Tonnage,is_train,Recolte,Summed_Irrigation,Total_N,Total_P,Total_K,Year,Month,Day
0,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10001,TR21010,1847.000000,True,0,37715.1,287.145757,281.399078,782.276993,2020,1,12
1,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10001,TR21010,2328.324585,True,1,57936.9,287.145757,281.399078,782.276993,2021,1,1
2,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10001,TR21010,7473.995360,True,2,75728.4,287.145757,281.399078,782.276993,2021,1,2
3,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10001,TR21010,19828.100000,True,3,106078.8,708.002903,373.085811,927.583113,2021,1,3
4,20/21,HAOUZ,Lalla Takerkoust,AF,AF1,10003,TR22050,15284.900000,True,0,13726.5,41.313328,189.180149,313.334136,2020,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3858,22/23,SOUSS,Ouled Abbou,SPA,SPA5,21021,TR21021,6338.000000,True,2,0.0,0.000000,0.000000,0.000000,2022,1,12
3871,22/23,SOUSS,Ouled Abbou,SPA,SPA7,21081,TR21081,650.000000,True,0,0.0,0.000000,0.000000,0.000000,2022,1,10
3872,22/23,SOUSS,Ouled Abbou,SPA,SPA7,21081,TR21081,4721.000000,True,1,0.0,0.000000,0.000000,0.000000,2022,1,11
3873,22/23,SOUSS,Ouled Abbou,SPA,SPA7,21081,TR21081,8721.800000,True,2,0.0,0.000000,0.000000,0.000000,2022,1,12


In [48]:
# Feature selection
features = train_test.drop(['Tonnage'], axis=1)
target = train_test['Tonnage']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [49]:
# Define a pipeline
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])),
    ('regressor', RandomForestRegressor())
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
predictions = pipeline.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, predictions))

Mean Squared Error: 1461486034.1500068


In [52]:
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})

In [53]:
results_df

,Actual,Predicted
1390,16943.00000,5576.004095
762,3182.30000,8237.349603
2252,24593.00000,9783.393132
11,30029.84951,9367.085721
3493,10040.00000,12139.258580
...,...,...
3327,4808.00000,13397.600660
1291,4832.13119,4068.197419
2526,35188.31036,4722.441612
2639,23951.30000,40246.647921
